In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
path = '/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv'
data = pd.read_csv(path)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
width,height=48,48
data1=data[data[' Usage']=='Training']
datapoints=data1[' pixels'].tolist()
data2=data[data[' Usage']=='PublicTest']
datapoints1=data2[' pixels'].tolist()

In [ ]:
len(datapoints),len(datapoints1)

In [ ]:
##getting x and y
X=[]
for xseq in datapoints:
    xx=[int(xp) for xp in xseq.split(' ')]
    xx=np.asarray(xx).reshape(width,height)
    X.append(xx.astype('float32'))
X=np.asarray(X)

X1=[]
for xseq in datapoints1:
    xx=[int(xp) for xp in xseq.split(' ')]
    xx=np.asarray(xx).reshape(width,height)
    X1.append(xx.astype('float32'))
X1=np.asarray(X1)


In [ ]:
X.shape,X1.shape

In [ ]:
y = pd.get_dummies(data1['emotion'])
y1=pd.get_dummies(data2['emotion'])

In [ ]:
y=y.values
y1=y1.values

In [ ]:
y.shape,y1.shape

In [ ]:
y[0],y1[0]

In [ ]:
import sys, os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

In [ ]:
X.shape,y.shape

HISTOGRAM EQUALIZER

In [ ]:
import cv2
x1=[]
for i in range(X.shape[0]):
    histimage=cv2.equalizeHist(X[i].astype('uint8'))
    x1.append(np.asarray(histimage).astype('float32'))
x1=np.asarray(x1).astype('float32')

x2=[]
for i in range(X1.shape[0]):
    histimage=cv2.equalizeHist(X1[i].astype('uint8'))
    x2.append(np.asarray(histimage).astype('float32'))
x2=np.asarray(x2).astype('float32')

print(x1.shape,x2.shape)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x1[1245],cmap='gray')

In [ ]:
plt.imshow(X[1245],cmap='gray')

In [ ]:
x1.shape,x2.shape

NORMALIZATION

In [ ]:
x1 -= np.mean(x1, axis=0)
x1 /= np.std(x1, axis=0)

x2-=np.mean(x2,axis=0)
x2/=np.std(x2,axis=0)

In [ ]:
y.shape,y1.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils


In [ ]:
x1.shape,x2.shape,y.shape,y1.shape

In [ ]:
x1 = x1.reshape(x1.shape[0], 48, 48, 1)

x2 = x2.reshape(x2.shape[0], 48, 48, 1)

In [ ]:
###model

model2 = Sequential()

model2.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model2.add(Conv2D(num_features, kernel_size=(5, 5), activation='relu', padding='same'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model2.add(Dropout(0.5))

model2.add(Conv2D(2*num_features, kernel_size=(5, 5), activation='relu', padding='same'))
model2.add(BatchNormalization())

model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model2.add(Dropout(0.5))


model2.add(Flatten())

model2.add(Dense(300, activation='relu'))
model2.add(Dropout(0.4))



model2.add(Dense(num_labels, activation='softmax'))

In [ ]:
model2.summary()

In [ ]:
model2.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

In [ ]:
history2 = model2.fit(np.array(x1), np.array(y),
          batch_size=batch_size,
          epochs=100,
          verbose=1,
          validation_data=(np.array(x2), np.array(y1)),
          shuffle=True)

In [ ]:
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
fer_json = model2.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model2.save_weights("fer.h5")
print("Saved model to disk")

In [ ]:
y_pred=model2.predict(x2)
y_pred.shape,y1.shape

In [ ]:
y_pred.shape

In [ ]:
rounded_predictions = model2.predict_classes(x2)
rounded_predictions

In [ ]:
rounded_labels=np.argmax(y1, axis=1)
rounded_labels

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
cm = confusion_matrix(rounded_labels, rounded_predictions)
cm

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
plot_confusion_matrix(conf_mat=cm,show_normed=True,figsize=(7,7))
plt.xticks(tick_marks, labels, rotation=45)
plt.yticks(tick_marks, labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


cm = confusion_matrix(rounded_labels, rounded_predictions)
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
title='Confusion matrix'
print(cm)

plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title(title)
plt.colorbar()
tick_marks = np.arange(len(labels))
plt.xticks(tick_marks, labels, rotation=45)
plt.yticks(tick_marks, labels)
fmt = 'd'
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()